# Expectation Maximisation
One of the recent papers on the one of hottest deep learning ideas around right now, capsules or CapsNet, uses a classic machine learning method, Expectation Maximisation, as part of its architecture. I have been revisiting probabilistic machine learning and Bayesian methods recently to gain a better understanding of deep learning so I thought it would be a good idea to write blogpost on this topic in the context of the CapsNet architecture.

The expectation maximisation (EM) algorithm is a core topic in most machine learning course so there are plenty of explanations around. This is intended to be mathematical post intended for those trying who want to get to grips with the details of the algorithm, which can be quite tricky although not so bad once you get used it. So whilst a background in probability theory, calculus and a bit of linear algebra is assumed, I will try to go through the steps as clearly as possible.

Essentially EM is an optimisation method that can be used to find maximum likelihood (ML) or maximum a posterior (MAP) estimates of the parameters of a model. 

[Define ML and MAP]

To understand why we might need it, suppose that we have a set of data points $\mathbf{x}^{(i)}$ and latent variables $z^{(i)}$ on which the data points depend. One way this could work is if we model the data generation process as follows:

1. First draw $z$ from some distribution $p(z\rvert\phi)$. 
2. Suppose that $z = k$ where $k = 1,2,...,K$. 
3. The value of $z$ determines the parameters $\pi_k$ for a distribution over $x$
4. Now draw $x$ from $p(x|\pi_k)$

Let us see how this works with the CapsNet architecture. We have two layers of capsules layer $L$ and layer $L+1$. 

Each capsule comprises a pose and an activation probabilty, which we assumed have already been decided. Now we need to decide which capsules to activate in layer $L+1$ and how to assign a capsule in layer $L$ to one in layer $L+1$. 

We approach this like a mixture of Gaussians problem. 

[Give example of mixture of Gaussians]

Here the datapoints are the active capsules in layer $L$ and the Gaussians are the capsules in layer $L+1$. 

Let us first compare the problem to fitting a mixture of Gaussians.

- In GMM we have data points $x$, latent variables $z$ and parameters $\theta$ where $z$ determines the parameters of the Gaussian to which $x$ belongs out of $K$ possible Gaussians.
- Here the votes $V_{ic}$ from the capsules are analogous to the datapoints
- There are c possible Gaussians to which the capsule may be assigned 
- The latent variable is the capsule c to which capsule i should be coupled
- The probability that a capsule should be coupled to capsule c is given by $a_c$.

We have two choices when deciding whether or not to activate a higher layer capsule:

1. If we don't activate it then we need to describe the means of all the lower level capsules that are assigned to the higher level one and have to pay a fixed cost per lower level capsule in order to do so.
2. If we do activate it then a cost is incurred for coding its mean and the fact that it is active. We also need to pay costs for describing the discrepancies between the lower-level means and the values that we get by predicting them from 




This is summarised in the diagram below. We shall see specific examples later. Our goal is to find a distribution over the examples $x$ that accounts for the latent variables. The log likelihood of the observed data can be written as:

    
$$l(\mathbf{\theta}) = \sum_{i=1}^{N} \log p(\mathbf{x}^{(i)}\rvert\mathbf{\theta}) 
    = \sum_{i=1}^{N}\log\left[\sum_{\mathbf{z^{(i)}}}p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})\right]$$
    
where $\mathbf{\theta}$ represents all the parameters of the distributions for $z$ and $x$.

[Give a few examples]

As we have a sum of terms within the $\log$, we cannot in general optimise $l(\mathbf{\theta})$ with respect to $\mathbf{\theta}$ either analytically or by using optimisation methods to find global maximum.

## The steps of the EM algorithm

The EM algorithm tackles the optimisation problem by constructing a new function called an auxiliary function that can be optimised either analytically or by using standard optimisation methods. The algorithm iteratively calculates and optimises the auxiliary function, so that after $t$ iterations of the algorithm we have a set of parameters $\theta^{t-1}$. 

To estimate the parameters using EM, first consider the situation where we know what these latent variables $z$ are. For example suppose that for each $x^{(i)}$ we know what value of $k = 1,2,...,K$. This would enable us to write down the complete data log likelihood, which is a joint distribution over $z$ and $x$:

$$l_c(\mathbf{\theta}) = \sum_{i=1}^{N}\log p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})$$

Of course we can't do this but instead, during each iteration of the algorithm, we take the expected value of this conditioned on the data, denoted as $D$, and the parameters in the previous iteration $\mathbf{\theta}^{t-1}$:

$$Q(\mathbf{\theta}, \mathbf{\theta}^{t-1}) = E[l_c(\mathbf{\theta})\rvert D,\mathbf{\theta}^{t-1}]$$

This function is known as the auxiliary function. It needs to be a function of the variable $\mathbf{\theta}$, since we want optimise with respect to this, as well as its estimate after $t$ iterations $\mathbf{\theta}^{t-1}$ since we need this to take the expectation.

The step of calculating the expectation $Q$ is called the E-step, whilst the step of maximising $Q$ with respect to the parameters is known as the M-step. 

[State variation for MAP]

[Give a few examples]

# How EM works 

Our original goal was to find the ML or MAP estimates for the observed data likelihood function so we need to show that the parameters found by the EM-algorithm will converge to the true estimates. To do so, we need to rely on Jensen's inequality. 

Let's now understand where the auxiliary function that we optimise instead of the log-likelihood comes from. We can write the observed data log-likelihood as follows

$$l(\mathbf{\theta}) = \sum_{i=1}^{N} \log p(\mathbf{x}^{(i)}\rvert\mathbf{\theta}) 
    = \sum_{i=1}^{N}\log\left[\sum_{\mathbf{z^{(i)}}}p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})\right]
    = \sum_{i=1}^{N}\log\left[\sum_{\mathbf{z^{(i)}}} q(\mathbf{z}_i)  \frac{p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})}{q(\mathbf{z}_i) }\right]$$
    
where the term $q(\mathbf{z}_i)$ is some arbitrary distribution.

Jensen's inequality states that for a concave function $f(x)$, $f(E[x]) \geq E[f(x)]$. Here since log(x) is a concave function, each term in the sum over the examples $i$ is lower bounded as follows:

$$\log\left[\sum_{\mathbf{z^{(i)}}} q(\mathbf{z}_i)  \frac{p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})}{q(\mathbf{z}_i) }\right] = \log\left(E_{z_i \sim q(\mathbf{z}_i)}\left[\frac{p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})}{q(\mathbf{z}_i) }\right]\right) \geq E_{z_i \sim q(\mathbf{z}_i)}\left[\log\left(\frac{p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})}{q(\mathbf{z}_i) }\right)\right]$$

Thus the observed data log-likelihood has the following lower bound:

$$l(\mathbf{\theta})\geq \underset{i}{\sum} E_{z_i \sim q(\mathbf{z}_i)}\left[\log\left(p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})\right)\right]$$


Since Jensen's inequality applies to expectations in general it holds for expectations using *any* distribution. 
We would like to maximise $l(\mathbf{\theta})$ so it makes sense to choose a distribution that will give us the highest possible lower bound, which would be equality with $l(\mathbf{\theta})$. So we choose a distribution which makes the lower bound as high as possible for a given set of parameters. From the inequality above we know that the highest possible value the lower bound can assume, given parameters $\mathbf{\theta}$, is $l(\mathbf{\theta})$ itself. Denoting the lower bound as $L(\mathbf{\theta}, q_i)$, we get:

$$L(\mathbf{\theta}, q_i)=\underset{i}{\sum} E_{\mathbf{z}_i \sim q_i(\mathbf{z}_i)}\left[\log\left(\frac{p(\mathbf{x}^{(i)}, \mathbf{z}^{(i)}\rvert\mathbf{\theta})}{q_i(\mathbf{z}_i) }\right)\right]=\underset{i}{\sum}\underset{\mathbf{z}_i}{\sum} q_i(\mathbf{z}_i)\log\left(\frac{p(\mathbf{z}^{(i)}\rvert \mathbf{x}^{(i)}, \mathbf{\theta})
p(\mathbf{x}^{(i)}\rvert  \mathbf{\theta})}{q_i(\mathbf{z}_i) }\right) = -KL\left(q_i(\mathbf{z}_i)\rvert p(\mathbf{z}^{(i)}\rvert \mathbf{x}^{(i)}, \mathbf{\theta})\right) + \log\left(p(\mathbf{x}^{(i)}\rvert  \mathbf{\theta})\right)$$

The KL-divergence is always non-negative which means that the distribution which leads to the largest value for $L(\mathbf{\theta}, q_i)$ is that which makes the KL-divergence 0 which happens when $q_i(\mathbf{z}_i)  = p(\mathbf{z}^{(i)}\rvert \mathbf{x}^{(i)}, \mathbf{\theta})$. In this situation only $\log\left(p(\mathbf{x}^{(i)}\rvert  \mathbf{\theta})\right) = l(\theta)$ remains on the right-hand side making $L(\mathbf{\theta}, q_i) = l(\theta)$.

Now notice that the lower bound is a function of the parameters $\theta$. However we also need a specific set of parameter values in order to evaluate the expected value. Recall that in the M-step we find the optimal parameters of the lower bound. At every point these parameters are our best guess so far of the true parameters and we use these to evaluate $q_i(\mathbf{z}_i)  = p(\mathbf{z}^{(i)}\rvert \mathbf{x}^{(i)}, \mathbf{\theta})$. This is why $Q(\mathbf{\theta}, \mathbf{\theta}^{t-1})$ is a function of $\theta$ as well as the parameters found at the end of the previous iteration, $\theta^{t-1}$